<a href="https://colab.research.google.com/github/surajn581/covid19_detection/blob/master/data_augmentation_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
base_path = '/content/drive/My Drive/Colab Notebooks/covid19_detection'

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [20]:
tf.random.set_seed(12)

In [21]:
!ls "/content/drive/My Drive/Colab Notebooks/covid19_detection/data"

covid  healthy	noncovid


In [24]:
train_data_dir= "/content/drive/My Drive/Colab Notebooks/covid19_detection/data"

In [27]:
# Directing Images to train folder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_height, img_width=64,64
batch_size=16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.3,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

# Splitting images for validation set
validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 105 images belonging to 3 classes.
Found 45 images belonging to 3 classes.
